# Joint Training Hybrid Model
The second approach is to have the regression model to incorporate in its training and testing the result of the classification model. The objective is to incorporate the ground truth of of the classification data set into the training data of the regression data set and demonstrated, that is, allow the model to learn to predict the amplitude of a constraint, given that that constraint exists. 

During testing, the truthful constraint violation Boolean feature is substituted by the prediction of the classification model, and the 
model will predict the amplitude of the constraint violation, given the prediction of its occurrence. 

## Model selection
Considering the datasets benchmark, the best regression model for max_u and min_u are: 
- max_u: lr focused
- min_u: gb sparse

The best classification model for max_u and min_u are:
- max_u: xgb balanced
- min_u: gb sparse


In [1]:
import sys;sys.path.append('..');from thesis_package import aimodels as my_ai, utils, metrics
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import os

d:\IST\Thesis\new_thesis\code\AI-to-forecast-constraints-in-the-energy-systems\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [2]:
exogenous_data = pd.read_csv('..\data\processed\production\exogenous_data_extended.csv').drop(columns=['date'])

In [3]:
# Classification data balanced
y_max_u_balanced_class = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_bool_constr.csv')
exogenous_data_balanced_max_u = pd.read_csv('..\data\ground_truth\exogenous_data_vm_pu_max_balanced.csv').drop(columns=['date'])
y_min_u_balanced_class = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_bool_constr.csv')
exogenous_data_balanced_min_u = pd.read_csv('..\data\ground_truth\exogenous_data_vm_pu_min_balanced.csv').drop(columns=['date'])
y_max_u_balanced_class = y_max_u_balanced_class[utils.cols_with_positive_values(y_max_u_balanced_class)]
y_min_u_balanced_class = y_min_u_balanced_class[utils.cols_with_positive_values(y_min_u_balanced_class)]

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data_balanced_max_u, y_max_u_balanced_class, test_size=0.2, scaling=True)
data_max_u_bool_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data_balanced_min_u, y_min_u_balanced_class, test_size=0.2, scaling=True)
data_min_u_bool_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

In [4]:
# Regresison data balanced
y_max_u_balanced = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_constr.csv')
exogenous_data_balanced_max_u = pd.read_csv('..\data\ground_truth\exogenous_data_vm_pu_max_balanced.csv').drop(columns=['date'])
y_min_u_balanced = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_constr.csv')
exogenous_data_balanced_min_u = pd.read_csv('..\data\ground_truth\exogenous_data_vm_pu_min_balanced.csv').drop(columns=['date'])

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data_balanced_max_u, y_max_u_balanced, test_size=0.2, scaling=True)
data_max_u_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data_balanced_min_u, y_min_u_balanced, test_size=0.2, scaling=True)
data_min_u_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

In [5]:
# Classification data sparse
y_max_u_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_sparse_bool_constr.csv').drop(columns=['timestamps'])
y_min_u_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_sparse_bool_constr.csv').drop(columns=['timestamps'])
y_max_u_bool = y_max_u_bool[utils.cols_with_positive_values(y_max_u_bool)]
y_min_u_bool = y_min_u_bool[utils.cols_with_positive_values(y_min_u_bool)]

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u_bool, test_size=0.2, scaling=True)
data_max_u_bool = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u_bool, test_size=0.2, scaling=True)
data_min_u_bool = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

In [6]:
# Regression data sparse
y_max_u_sparse = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_constr.csv').drop(columns=['timestamps'])
y_min_u_sparse = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_constr.csv').drop(columns=['timestamps'])

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u_sparse, test_size=0.2, scaling=True)
data_max_u_sparse = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u_sparse, test_size=0.2, scaling=True)
data_min_u_sparse = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}

In [7]:
max_u_threshold = utils.compute_threshold(y_max_u_sparse)
min_u_threshold = utils.compute_threshold(y_min_u_sparse)

## Prepared Hybrid data sets

### Training


In [8]:
# max u
data_max_u_hybrid = {}
data_max_u_hybrid['X_train'] = pd.concat([data_max_u_balanced['X_train'], data_max_u_bool_balanced['y_train']], axis=1)
data_max_u_hybrid['y_train'] = deepcopy(data_max_u_balanced['y_train'])
# min u 
data_min_u_hybrid = {}
data_min_u_hybrid['X_train'] = pd.concat([data_min_u_sparse['X_train'], data_min_u_bool['y_train']], axis=1)
data_min_u_hybrid['y_train'] = deepcopy(data_min_u_sparse['y_train'])

### Testing

In [9]:
# max u
classifier_max_u_balanced = utils.deserialize_object('pickles\dataset_benchmark\max_u_classifier_balanced')
max_u_class_xgb = deepcopy(classifier_max_u_balanced.strategies[1])
print(classifier_max_u_balanced.strategies[1])
class_result_max_u = data_max_u_bool['y_test'] #max_u_class_xgb.predict(data_max_u_bool)
data_max_u_hybrid['X_test'] = pd.concat([data_max_u_sparse['X_test'], class_result_max_u], axis=1)
data_max_u_hybrid['y_test'] = deepcopy(data_max_u_sparse['y_test'])
# min u 
classifier_min_u_sparse = utils.deserialize_object('pickles\dataset_benchmark\min_u_classifier')
min_u_class_gb = deepcopy(classifier_min_u_sparse.strategies[1])
print(classifier_min_u_sparse.strategies[0])
class_result_min_u = data_min_u_bool['y_test']#min_u_class_gb.predict(data_min_u_bool)
data_min_u_hybrid['X_test'] = pd.concat([data_min_u_sparse['X_test'], class_result_min_u], axis=1)
data_min_u_hybrid['y_test'] = deepcopy(data_min_u_sparse['y_test'])

In [10]:
print('data_max_u_hybrid X_train shape: ', data_max_u_hybrid['X_train'].shape)
print('data_max_u_hybrid y_train shape: ', data_max_u_hybrid['y_train'].shape)
print('data_max_u_hybrid X_test shape: ', data_max_u_hybrid['X_test'].shape)
print('data_max_u_hybrid y_test shape: ', data_max_u_hybrid['y_test'].shape)
print('\n')
print('data_min_u_hybrid X_train shape: ', data_min_u_hybrid['X_train'].shape)
print('data_min_u_hybrid y_train shape: ', data_min_u_hybrid['y_train'].shape)
print('data_min_u_hybrid X_test shape: ', data_min_u_hybrid['X_test'].shape)
print('data_min_u_hybrid y_test shape: ', data_min_u_hybrid['y_test'].shape)


data_max_u_hybrid X_train shape:  (5561, 22)
data_max_u_hybrid y_train shape:  (5561, 34)
data_max_u_hybrid X_test shape:  (9044, 22)
data_max_u_hybrid y_test shape:  (9044, 34)


data_min_u_hybrid X_train shape:  (36172, 21)
data_min_u_hybrid y_train shape:  (36172, 34)
data_min_u_hybrid X_test shape:  (9044, 21)
data_min_u_hybrid y_test shape:  (9044, 34)


In [11]:
data_max_u_hybrid['X_train']

,T,R,wind_speed,wind_direction,season,weekday,last_hour_mean_wind_speed,last_day_mean_wind_direction,last_hour_mean_temperature,cos_hour_day,...,bus_9,bus_10,bus_11,bus_12,bus_13,bus_14,bus_15,bus_16,bus_18,bus_32
0,0.980411,0.275329,0.156478,0.948083,0.666667,0.571429,0.153765,0.970115,0.978607,-0.965926,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.966654,0.814006,0.630440,0.968780,0.666667,0.571429,0.653422,0.967337,0.967426,-0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.981300,0.346816,0.206103,0.905242,0.666667,0.285714,0.211845,0.647530,0.979773,-1.000000,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,0.974025,0.576971,0.377461,0.962206,0.666667,0.857143,0.389402,0.967836,0.974051,-0.965926,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,0.964664,0.038244,0.247086,0.047683,0.333333,0.714286,0.259155,0.041805,0.964740,0.866025,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5556,0.947001,0.334747,0.071293,0.818253,0.000000,0.142857,0.078117,0.771496,0.948084,-0.258819,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
5557,0.949473,0.228576,0.464368,0.472328,1.000000,0.857143,0.479484,0.465658,0.949584,-1.000000,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
5558,0.946500,0.167410,0.142277,0.700680,1.000000,0.714286,0.165313,0.731423,0.945704,-1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5559,0.936309,0.012011,0.410154,0.441789,1.000000,0.857143,0.422732,0.449702,0.935937,-0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Training

In [12]:
balanced_hyper_params = {}
sparse_hyper_params = {}
for file in os.listdir('hyper_params_results_mcc'):
    if file.endswith('.csv') and 'regression_sparse' in file:
        df = pd.read_csv(os.path.join('hyper_params_results', file))
        sparse_hyper_params[file] = df
    elif file.endswith('.csv') and 'regression_balanced' in file:
        df = pd.read_csv(os.path.join('hyper_params_results', file))
        balanced_hyper_params[file] = df
    else:
        pass
import ast
def get_hyper_params_from_df(df):
    output = {}
    for row in df.iterrows():
        if row[1]['params'] != 'value':
            try:
                output[row[1]['params']] = ast.literal_eval(row[1]['value'])
            except :
                output[row[1]['params']] = row[1]['value']
    return output


# max_u
if 'hybrid_regressor_max_u.pickle' not in os.listdir('pickles\hybrid_models_benchmark'):
    print('training max u hybrid model... wait for it...')
    hyper_params = get_hyper_params_from_df(balanced_hyper_params['params_gradient_boost_regression_balanced_max_u.csv'])
    hybrid_regressor_max_u = my_ai.Context(strategy=my_ai.GradientBoostRegressorStrategy(hyper_params)) 
    hybrid_regressor_max_u.fit(data=data_max_u_hybrid)
    utils.serialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_max_u', hybrid_regressor_max_u)
else: 
    print('loading max u hybrid model...')
    hybrid_regressor_max_u = utils.deserialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_max_u')

# min_u 
if 'hybrid_regressor_min_u.pickle' not in os.listdir('pickles\hybrid_models_benchmark'):
    hyper_params = get_hyper_params_from_df(sparse_hyper_params['params_gradient_boost_regression_sparse_min_u.csv'])
    hybrid_regressor_min_u = my_ai.Context(my_ai.GradientBoostRegressorStrategy(hyper_params))
    hybrid_regressor_min_u.fit(data=data_min_u_hybrid)
    utils.serialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_min_u', hybrid_regressor_min_u)
else:
    print('loading min u hybrid model...')
    hybrid_regressor_min_u = utils.deserialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_min_u')

loading max u hybrid model...
loading min u hybrid model...


## Testing

In [19]:
# Load models
# max_u
regression_max_u_focused = utils.deserialize_object('pickles\dataset_benchmark\max_u_regressor_focused')
max_u_reg_lr = deepcopy(regression_max_u_focused.strategies[0])
print(max_u_reg_lr)
reg_result = max_u_reg_lr.predict(data_max_u_sparse)

In [20]:
reg_result

,ext_grid,bus_1,bus_2,bus_3,bus_4,bus_5,bus_6,bus_7,bus_8,bus_9,...,bus_30,bus_31,bus_17,bus_21,bus_24,bus_18,bus_23,bus_27,bus_32,bus_33
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
_threshold = lambda experiment: max_u_threshold / data_max_u_sparse['scaler']['y'] if 'max_u' in experiment else min_u_threshold/ data_min_u_sparse['scaler']['y']
scaled_threshold = _threshold('max_u')
metric = metrics.Metrics()
metric.get_prediction_scores(pd.DataFrame(reg_result['bus_16']), pd.DataFrame(data_max_u_sparse['y_test']['bus_16']), threshold=scaled_threshold)

Threshold:  0.05260425830867303
true_positives_sse:  25.70673758581787
self.true_positives_rmse:  0.19587818932555892
self.hybrid_true_positives_rmse:  538.7616131518755
self.true_positives_ctr:  670


In [26]:
#pd.DataFrame(data_max_u_sparse['y_test']['bus_16'])
scaled_threshold

0.05260425830867303

In [14]:
# max_u
metric = metrics.Metrics()
_threshold = lambda experiment: max_u_threshold / data_max_u_sparse['scaler']['y'] if 'max_u' in experiment else min_u_threshold/ data_min_u_sparse['scaler']['y']
scaled_threshold = _threshold('max_u')
metric.get_prediction_scores(reg_result, data_max_u_sparse['y_test'], threshold=scaled_threshold)
reg_accuracy = metric.hybrid_accuracy
reg_precision = metric.hybrid_precision
reg_recall = metric.hybrid_recall
reg_f1 = metric.hybrid_f1
reg_mcc = metric.hybrid_mcc
reg_evaluation = pd.Series({'accuracy': reg_accuracy, 'precision': reg_precision, 'recall': reg_recall, 'f1': reg_f1, 'mcc': reg_mcc})
# same for hybrid model 
# max_u
hybrid_reg_result = hybrid_regressor_max_u.predict(data_max_u_hybrid)
metric.get_prediction_scores(hybrid_reg_result, data_max_u_hybrid['y_test'], threshold=scaled_threshold)
hybrid_reg_accuracy = metric.hybrid_accuracy
hybrid_reg_precision = metric.hybrid_precision
hybrid_reg_recall = metric.hybrid_recall
hybrid_reg_f1 = metric.hybrid_f1
hybrid_reg_mcc = metric.hybrid_mcc
hybrid_reg_evaluation = pd.Series({'accuracy': hybrid_reg_accuracy, 'precision': hybrid_reg_precision, 'recall': hybrid_reg_recall, 'f1': hybrid_reg_f1, 'mcc': hybrid_reg_mcc})
pd.DataFrame([reg_evaluation, hybrid_reg_evaluation], index=['regression', 'hybrid'])


Threshold:  0.05260425830867303
true_positives_sse:  134.6944365960727
self.true_positives_rmse:  0.17655608734280376
self.hybrid_true_positives_rmse:  3558.101146591745
self.true_positives_ctr:  4321
Threshold:  0.05260425830867303
true_positives_sse:  203.41872868892142
self.true_positives_rmse:  0.22285146523821356
self.hybrid_true_positives_rmse:  3183.2003983842774
self.true_positives_ctr:  4096


,accuracy,precision,recall,f1,mcc
regression,0.977467,0.366413,0.822844,0.507041,0.540358
hybrid,0.977352,0.349136,0.761696,0.478805,0.506539


In [15]:
def experiment_string(value):
    output = ''
    if 'reg' in value:
        output += 'Regression'
    elif 'hybrid' in value:
        output += 'Hybrid'
    return output
def print_df_in_latex(df):
    for index, row in df.iterrows():
        print( experiment_string(index) + ' & ' + str(round(row['accuracy'], 2)) + ' & ' + str(round(row['precision'], 2)) + ' & ' + str(round(row['recall'], 2)) + ' & ' + str(round(row['f1'], 2)) +' & ' + str(round(row['mcc'], 2)) + ' \\\\')
print_df_in_latex(pd.DataFrame([reg_evaluation, hybrid_reg_evaluation], index=['regression', 'hybrid']))

Regression & 0.98 & 0.37 & 0.82 & 0.51 & 0.54 \\
Hybrid & 0.98 & 0.35 & 0.76 & 0.48 & 0.51 \\


In [16]:
# min_u
regression_min_u_sparse = utils.deserialize_object('pickles\dataset_benchmark\min_u_regressor_sparse')
min_u_reg_gb = deepcopy(regression_min_u_sparse.strategies[1])
print(min_u_reg_gb)
reg_result = min_u_reg_gb.predict(data_min_u_sparse)

In [17]:
# min_u
scaled_threshold = _threshold('min_u')
metric.get_prediction_scores(reg_result, data_min_u_sparse['y_test'], threshold=scaled_threshold)
reg_accuracy = metric.hybrid_accuracy
reg_precision = metric.hybrid_precision
reg_recall = metric.hybrid_recall
reg_f1 = metric.hybrid_f1
reg_mcc = metric.hybrid_mcc
reg_evaluation = pd.Series({'recall': reg_recall, 'precision': reg_precision, 'f1': reg_f1, 'accuracy': reg_accuracy, 'mcc': reg_mcc})
# same for hybrid model 
# min_u
hybrid_reg_result = hybrid_regressor_min_u.predict(data_min_u_hybrid)
metric.get_prediction_scores(hybrid_reg_result, data_min_u_hybrid['y_test'], threshold=scaled_threshold)
hybrid_reg_accuracy = metric.hybrid_accuracy
hybrid_reg_precision = metric.hybrid_precision
hybrid_reg_recall = metric.hybrid_recall
hybrid_reg_f1 = metric.hybrid_f1
hybrid_reg_mcc = metric.hybrid_mcc
hybrid_reg_evaluation = pd.Series({'accuracy': hybrid_reg_accuracy, 'precision': hybrid_reg_precision, 'recall': hybrid_reg_recall, 'f1': hybrid_reg_f1, 'mcc': hybrid_reg_mcc})
pd.DataFrame([reg_evaluation, hybrid_reg_evaluation], index=['regression', 'hybrid'])

Threshold:  0.0644069763149719
true_positives_sse:  128.81555487840572
self.true_positives_rmse:  0.1598865660599132
self.hybrid_true_positives_rmse:  4233.331593624098
self.true_positives_ctr:  5039
Threshold:  0.0644069763149719
true_positives_sse:  128.81720105215922
self.true_positives_rmse:  0.15998286283904747
self.hybrid_true_positives_rmse:  4227.806251331074
self.true_positives_ctr:  5033


,recall,precision,f1,accuracy,mcc
regression,0.787945,0.353983,0.488506,0.970974,0.516571
hybrid,0.786738,0.354551,0.488814,0.971048,0.516597


In [18]:
print_df_in_latex(pd.DataFrame([reg_evaluation, hybrid_reg_evaluation], index=['regression', 'hybrid']))

Regression & 0.97 & 0.35 & 0.79 & 0.49 & 0.52 \\
Hybrid & 0.97 & 0.35 & 0.79 & 0.49 & 0.52 \\
